In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# import folium
# from folium.plugins import HeatMap
from scipy import stats
from ipyleaflet import Map, Marker, CircleMarker
from ipywidgets import SelectionSlider, VBox, interact
import pandas as pd

# Loading the datasets
sales_data = pd.read_csv('data/sales_data.csv')
customer_data = pd.read_csv('data/customer_data.csv')
shopping_mall_data = pd.read_csv('data/shopping_mall_data.csv')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, VBox, Output

# 示例数据
data = {
    "x": ["A", "A", "A", "B", "B", "B", "C", "C", "C"],
    "date": pd.date_range(start="2021-01-01", periods=9),
    "price": [100, 110, 105, 200, 190, 195, 300, 310, 305]
}
df = pd.DataFrame(data)

# 输出控件
output = Output()

# 滑动平均函数
def moving_average(data, window_size):
    return data.rolling(window=window_size, center=True).mean()

# 绘制函数
def plot_all_categories(window_size):
    with output:
        output.clear_output(wait=True)
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        categories = df["x"].unique()
        for category in categories:
            data_cat = df[df["x"] == category]
            data_cat = data_cat.sort_values("date")  # 确保按日期排序
            
            # 原始价格
            ax.plot(data_cat["date"], data_cat["price"], label=f"{category} - Original", alpha=0.5)
            
            # 滑动平均价格
            smoothed_price = moving_average(data_cat["price"], window_size)
            ax.plot(data_cat["date"], smoothed_price, label=f"{category} - Smoothed (window={window_size})")
        
        ax.set_title("Price vs Date (All Categories)")
        ax.set_xlabel("Date")
        ax.set_ylabel("Price")
        ax.legend()
        ax.grid(True)
        
        plt.tight_layout()
        plt.show()

# 滑块
slider = IntSlider(
    value=3,
    min=1,
    max=7,
    step=1,
    description="Window Size:"
)

# 滑块绑定事件
def on_slider_change(change):
    plot_all_categories(change["new"])

slider.observe(on_slider_change, names="value")

# 初始化绘图
plot_all_categories(slider.value)

# 显示控件
VBox([slider, output])
